In [18]:
import numpy as np
import pandas as pd
from pathlib import Path

In [19]:
#상수 모음

encoding  = 'cp949'

def make_path(name, mode):
    if mode not in ['원본', '전처리필요', '분석용']:
        raise Exception('모드 잘못 입력')
    return mode + '데이터/' + name + '.csv'

use_p = ['당현', '월계', '화랑', '노원역', '상계1', '마들']

X_DIFF_M = 0.00001125
Y_DIFF_M = 0.00000898
POLICE_RAD_M = 900
GRID_DIFF_M  = 90

SAMPLE = 5

In [20]:
def police_location_csv():
    p_loc = pd.read_csv(make_path('경찰서정보', '원본'), encoding=encoding)
    p_loc = p_loc.drop(['연번', '경찰서', '구분', '전화번호'], axis=1)
    p_loc = p_loc.loc[p_loc['관서명'].isin(use_p), :]
    p_loc.to_csv(make_path('경찰서정보', '전처리필요'), index=False)

police_location_csv()

In [21]:
def make_grid(p_x, p_y):
    # 정사각 격자점 생성
    x = np.linspace(p_x - X_DIFF_M*POLICE_RAD_M, p_x + X_DIFF_M*POLICE_RAD_M, 2*POLICE_RAD_M//GRID_DIFF_M + 1)
    y = np.linspace(p_y - Y_DIFF_M*POLICE_RAD_M, p_y + Y_DIFF_M*POLICE_RAD_M, 2*POLICE_RAD_M//GRID_DIFF_M + 1)
    X, Y = np.meshgrid(x, y)
    grid = np.stack([X, Y], axis = 2).reshape(-1, 2)

    # 원형 필터링
    dist = (grid - [p_x, p_y]) / [X_DIFF_M, Y_DIFF_M] # grid에는 경,위도가 담겼으므로 원점으로 옮기고 미터로 환산
    dist = np.linalg.norm(dist, axis = 1)
    grid = grid[dist <= POLICE_RAD_M]
    
    #데이터프레임화
    grid = pd.DataFrame(grid, columns=['경도', '위도'])
    return grid

def make_grid_csv():
    p_loc = pd.read_csv(make_path('경찰서정보', '분석용'))
    for i in p_loc.index:
        grid = make_grid(p_loc.loc[i, '경도'], p_loc.loc[i, '위도'])
        grid.to_csv(make_path('격자점/'+p_loc.loc[i, '이름'], '분석용'), index=False)

make_grid_csv()

In [22]:
def sample_square(df, size):
    #정사각범위 샘플링 -> 영역 내 겹치지 않고 빈 공간 없게 뽑기 위함
    sample = np.random.uniform(-GRID_DIFF_M/2, GRID_DIFF_M/2, size=size)
    sample *= np.array([X_DIFF_M, Y_DIFF_M])
    return df + sample

def sample_circle(df, size):
    #원형 샘플링 -> 빈 공간 있지만 격자점 영역 구분 확실히 함
    # (x, y) -> (x + rcos(a), y + rsin(a)) 이용
    sample = GRID_DIFF_M/2 * np.sqrt(np.random.uniform(0, 1, size=size[0]))
    angle = np.random.uniform(0, 2*np.pi, size=size[0])
    dx, dy = np.cos(angle), np.sin(angle)
    return df + np.stack([sample*dx*X_DIFF_M, sample*dy*Y_DIFF_M], axis = 1)
    
def sample_point(file):
    base = pd.read_csv(file)
    grid = [base]
    size = (len(base), 2)

    # SAMPLE번 뽑아 임시 저장
    for _ in range(SAMPLE):
        grid.append(sample_circle(base, size))

    # 결합 후 저장
    grid = pd.concat(grid, ignore_index=False)
    grid.to_csv(make_path('샘플링/'+file.name, '전처리필요'), index=False)

def make_sampled_csv():
    grid = Path('분석용데이터/격자점/')
    for i in grid.iterdir():
        sample_point(i)

make_sampled_csv()

In [26]:
import folium

df = pd.read_csv(make_path('격자점/당현', '분석용'))
df2 = pd.read_csv(make_path('샘플링/당현', '전처리필요'))

# 지도 중심을 서울(첫 번째 위치)로 설정
m = folium.Map(location=[df['위도'].mean(), df['경도'].mean()], zoom_start=15)

# 데이터프레임의 각 위치에 마커 추가
for idx, row in df.iterrows():
    folium.Circle(
        location=[row['위도'], row['경도']],
        radius = 10,
        color = '#FF0000'
    ).add_to(m)
for idx, row in df2.iterrows():
    folium.Circle(
        location=[row['위도'], row['경도']],
        radius = 5,
        color = '#0000FF'
    ).add_to(m)

# 지도 출력
#m
